# Primer experimento :

## Descripcion :
Cambiamos el tamaño del batch para reducir el error,la prueba que se va a realizar es entrenar 1 modelo por secuencia y luego contrastar los resultados contra las demas secuencias.

## Datos de entrada:

Se esta usando la secuencias individuales de la libreria VIDRILO.
Tambien para la transformacion inicial se usa la media de cada dataset que se vaya a usar como modelo , como es una prueba en la que no se va a optimizar el entrenamiento de cada secuencia la media solo se calcula sobre el conjunto con el que se va a entrenar. Ejemplo en el caso de entrenar el model sobre la primera secuencia , la media se calcula sobre esta.

## Ajustes del entrenamiento:

Numero de iteraciones : 35000
Tamaño del mini batch : 50

In [ ]:
import caffe
import lmdb
import numpy as np
import matplotlib.pyplot as plt
from caffe.proto import caffe_pb2
from caffe.io import datum_to_array, array_to_datum

In [2]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
%matplotlib inline

In [3]:
blob = caffe.proto.caffe_pb2.BlobProto()
data = open( '/vol/pfc/data/means/Sequence5_mean.binaryproto' , 'rb' ).read()
blob.ParseFromString(data)
arr = np.array( caffe.io.blobproto_to_array(blob) )
out = arr[0]
#np.save( '/vol/pfc/data/means/Sequence5_mean.npy', out )

In [4]:
def predict(image_in):
    net.blobs['data'].data[...] = image_in
    out = net.forward()
    top_k = net.blobs['prob'].data[0].flatten().argsort()[-1:-6:-1]
    output_prob = out['prob'][0]
    print 'predicted class is:', output_prob.argmax()
    return output_prob.argmax()
    

In [3]:
datum = caffe.proto.caffe_pb2.Datum()
caffe.set_mode_cpu()

def load_sequence(path):
    lmdb_env = lmdb.open(path)
    lmdb_txn = lmdb_env.begin()
    lmdb_cursor = lmdb_txn.cursor()
    return lmdb_cursor

def load_model(model_path):
    deploy_path="/vol/pfc/prototxt/sequence1/train_sequence_25_lmdb_deploy.prototxt"    
    #net = caffe.Net(deploy_path,model_path, caffe.TEST)
    net = caffe.Classifier(deploy_path, model_path,
                       mean=np.load('/vol/pfc/data/means/Sequence1_mean.npy').mean(1).mean(1),
                       channel_swap=(2,1,0),
                       raw_scale=255,
                          image_dims=(320, 240))
    return net
   
def load_transform(net,mean_path):
    transformer = caffe.io.Transformer({'data': net.blobs['data'].data.shape})
    transformer.set_mean('data', np.load(mean_path).mean(1).mean(1))
    transformer.set_transpose('data', (2,0,1))
    return transformer
    
    

In [4]:
sequences=["/vol/pfc/data/datasets/Sequence1_lmdb","/vol/pfc/data/datasets/Sequence2_lmdb"
           ,"/vol/pfc/data/datasets/Sequence3_lmdb","/vol/pfc/data/datasets/Sequence4_lmdb"
           ,"/vol/pfc/data/datasets/Sequence5_lmdb"]

models=["/mnt/models/train_25_batch/train_sequence_1_50_lmdb_iter_35000.caffemodel","/mnt/model/train_25_batch/train_sequence_2_50_lmdb_iter_35000.caffemodel"
       ,"/mnt/models/train_25_batch/train_sequence_3_50_lmdb_iter_35000.caffemodel","/mnt/model/train_25_batch/train_sequence_4_50_lmdb_iter_35000.caffemodel"
       ,"/mnt/models/train_50_batch/train_sequence_5_50_lmdb_iter_35000.caffemodel"]

means=["/vol/pfc/data/means/Sequence1_mean.npy","/vol/pfc/data/means/Sequence2_mean.npy","/vol/pfc/data/means/Sequence3_mean.npy"
       ,"/vol/pfc/data/means/Sequence4_mean.npy","/vol/pfc/data/means/Sequence5_mean.npy"]

deploys=["/vol/pfc/prototxt/sequence1/train_sequence_25_lmdb_deploy.prototxt","/vol/pfc/prototxt/sequence2/train_sequence_25_lmdb_deploy.prototxt"
        ,"/vol/pfc/prototxt/sequence3/train_sequence_25_lmdb_deploy.prototxt","/vol/pfc/prototxt/sequence4/train_sequence_25_lmdb_deploy.prototxt"
        ,"/vol/pfc/prototxt/sequence5/train_sequence_25_lmdb_deploy.prototxt"]

In [6]:
def process_sequence(net,sequence,transform):
    data_array = []
    for key, value in sequence:
        datum.ParseFromString(value)
        label = datum.label
        data = caffe.io.datum_to_array(datum)
        image = np.transpose(data,(1, 2, 0))
        #prediccion = predict(transform.preprocess('data', image))
        prediction = net.predict([transform.preprocess('data', image)])
        
        data_array.append((label,prediccion)) 
        print " label "+str(label)+" preiccion "+str(prediccion)
    
    return data_array
        

In [ ]:

len_model=len(models)
len_sequences=len(sequences)
len_model=2
len_sequences=2
data_evaluate=[[]]
for i in range(0,len_model-1):
    model=models[i]
    print 'evalua el  modelo: '+str(model)
    net=load_model(model)
    transform=load_transform(net,means[i])
    
    for j in range(0,len_sequences-1):
        print 'evalua sequence '+str(sequences[j])
        data=load_sequence(sequences[j])
        data_evaluate=process_sequence(net,data,transform)
    


In [ ]:
data_evaluate